In [24]:
pip install ffmpeg

  Preparing metadata (setup.py) ... done
  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6084 sha256=0a7fc9a13aab65156ce02028888e96d704750e6540763c5e29df9e30b7e71769
  Stored in directory: /Users/stupam2/Library/Caches/pip/wheels/1d/57/24/4eff6a03a9ea0e647568e8a5a0546cdf957e3cf005372c0245
Successfully built ffmpeg
Note: you may need to restart the kernel to use updated packages.


In [25]:
# Import Required Libraries
from fastapi import FastAPI, UploadFile, File, HTTPException
import cv2
import torch
import numpy as np
from torchvision.transforms import Compose, Resize, ToTensor, Normalize
from PIL import Image
import logging
import os
import subprocess


In [26]:
# Initialize Logging and Load the MiDaS Model
# Set up logging
logging.basicConfig(level=logging.INFO)

# Load the MiDaS model from PyTorch Hub
model_type = "DPT_Hybrid"  # Specify the model type; can also be DPT_Large, DPT_Hybrid, etc.
model = torch.hub.load("intel-isl/MiDaS", model_type)
model.eval()
device = torch.device('cpu')  # Use CPU
model.to(device)


Using cache found in /Users/stupam2/.cache/torch/hub/intel-isl_MiDaS_master
/opt/homebrew/Caskroom/miniconda/base/envs/chatbot_env/lib/python3.9/site-packages/timm/models/_factory.py:117: UserWarning: Mapping deprecated model name vit_base_resnet50_384 to current vit_base_r50_s16_384.orig_in21k_ft_in1k.
  model = create_fn(


DPTDepthModel(
  (pretrained): Module(
    (model): VisionTransformer(
      (patch_embed): HybridEmbed(
        (backbone): ResNetV2(
          (stem): Sequential(
            (conv): StdConv2dSame(3, 64, kernel_size=(7, 7), stride=(2, 2), bias=False)
            (norm): GroupNormAct(
              32, 64, eps=1e-05, affine=True
              (drop): Identity()
              (act): ReLU(inplace=True)
            )
            (pool): MaxPool2dSame(kernel_size=(3, 3), stride=(2, 2), padding=(0, 0), dilation=(1, 1), ceil_mode=False)
          )
          (stages): Sequential(
            (0): ResNetStage(
              (blocks): Sequential(
                (0): Bottleneck(
                  (downsample): DownsampleConv(
                    (conv): StdConv2dSame(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
                    (norm): GroupNormAct(
                      32, 256, eps=1e-05, affine=True
                      (drop): Identity()
                      (act): Identit

In [27]:
# Define the Image Transformation
# Define the transformation required for the MiDaS input
transform = Compose([
    Resize((384, 384)),  # Resize to 384x384 pixels
    ToTensor(),  # Convert the image to a tensor
    Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])


In [28]:
# Define the Depth Estimation Function
def estimate_depth(frame):
    """Estimate the depth of a frame using the MiDaS model."""
    try:
        pil_image = Image.fromarray(frame)
        input_tensor = transform(pil_image).unsqueeze(0).to(device)
        with torch.no_grad():
            depth_map = model(input_tensor)
        # Get the original size to resize depth map
        original_size = (frame.shape[1], frame.shape[0])  # (width, height)
        depth_map = torch.nn.functional.interpolate(
            depth_map.unsqueeze(1),
            size=original_size,  # Resize depth map to original frame size
            mode='bicubic',
            align_corners=False
        ).squeeze().cpu().numpy()
        return depth_map
    except Exception as e:
        logging.error(f"Error estimating depth: {e}")
        raise


In [29]:
# Define the Function to Generate the Depth Video
def generate_depth_video(video_path, output_path):
    try:
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            logging.error("Error: Could not open video file.")
            return

        frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = cap.get(cv2.CAP_PROP_FPS)

        logging.info(f"Processing {frame_count} frames at {fps} FPS with resolution {frame_width}x{frame_height}.")

        # Define the codec and create VideoWriter object for temporary raw output
        temp_raw_path = "temp_raw.avi"
        raw_out = cv2.VideoWriter(temp_raw_path, cv2.VideoWriter_fourcc(*'MJPG'), fps, (frame_width, frame_height), isColor=True)

        processed_frames = 0
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                logging.info("Finished processing video.")
                break
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            depth_map = estimate_depth(frame_rgb)
            depth_map_normalized = cv2.normalize(depth_map, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
            depth_colormap = cv2.applyColorMap(depth_map_normalized, cv2.COLORMAP_JET)
            raw_out.write(depth_colormap)
            processed_frames += 1

            # Log every 10th frame to track progress
            if processed_frames % 10 == 0:
                logging.info(f"Processed {processed_frames}/{frame_count} frames.")

        cap.release()
        raw_out.release()

        logging.info(f"Processed {processed_frames} frames.")

        # Verify the raw output file size
        if os.path.exists(temp_raw_path) and os.path.getsize(temp_raw_path) > 0:
            # Use ffmpeg to convert raw video to QuickTime compatible format
            command = [
                'ffmpeg', '-y', '-i', temp_raw_path,
                '-c:v', 'libx264', '-pix_fmt', 'yuv420p',
                output_path
            ]
            try:
                subprocess.run(command, check=True)
                logging.info(f"Converted video saved to {output_path}.")
            except subprocess.CalledProcessError as e:
                logging.error(f"FFmpeg conversion failed: {e}")
                raise HTTPException(status_code=500, detail=f"FFmpeg conversion failed: {e}")
        else:
            logging.error("No frames were written to the temporary raw file.")
            raise HTTPException(status_code=500, detail="Failed to write frames to raw video.")

    except Exception as e:
        logging.error(f"Error generating depth video: {e}")
        raise
    finally:
        if os.path.exists(temp_raw_path):
            os.remove(temp_raw_path)


In [ ]:
# Define the Function to Generate the Depth Video
def generate_depth_video(video_path, output_path):
    try:
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            logging.error("Error: Could not open video file.")
            return

        frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = cap.get(cv2.CAP_PROP_FPS)

        logging.info(f"Processing {frame_count} frames at {fps} FPS with resolution {frame_width}x{frame_height}.")

        # Define the codec and create VideoWriter object for temporary raw output
        temp_raw_path = "temp_raw.avi"
        raw_out = cv2.VideoWriter(temp_raw_path, cv2.VideoWriter_fourcc(*'XVID'), fps, (frame_width, frame_height), isColor=True)

        processed_frames = 0
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                logging.info("Finished processing video.")
                break
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            depth_map = estimate_depth(frame_rgb)
            depth_map_normalized = cv2.normalize(depth_map, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
            depth_colormap = cv2.applyColorMap(depth_map_normalized, cv2.COLORMAP_JET)
            raw_out.write(depth_colormap)
            processed_frames += 1

            # Log every 10th frame to track progress
            if processed_frames % 10 == 0:
                logging.info(f"Processed {processed_frames}/{frame_count} frames.")
                
            # Debugging: Save a few frames as images to verify
            if processed_frames < 5:  # Save first 5 frames
                cv2.imwrite(f"debug_frame_{processed_frames}.png", depth_colormap)

        cap.release()
        raw_out.release()

        logging.info(f"Processed {processed_frames} frames.")

        # Verify the raw output file size
        if os.path.exists(temp_raw_path) and os.path.getsize(temp_raw_path) > 0:
            # Use ffmpeg to convert raw video to QuickTime compatible format
            command = [
                'ffmpeg', '-y', '-i', temp_raw_path,
                '-c:v', 'libx264', '-pix_fmt', 'yuv420p',
                output_path
            ]
            try:
                result = subprocess.run(command, check=True, stderr=subprocess.PIPE, text=True)
                logging.info(f"Converted video saved to {output_path}.")
            except subprocess.CalledProcessError as e:
                logging.error(f"FFmpeg conversion failed: {e.stderr}")
                raise HTTPException(status_code=500, detail=f"FFmpeg conversion failed: {e.stderr}")
        else:
            logging.error("No frames were written to the temporary raw file.")
            raise HTTPException(status_code=500, detail="Failed to write frames to raw video.")

    except Exception as e:
        logging.error(f"Error generating depth video: {e}")
        raise
    finally:
        if os.path.exists(temp_raw_path):
            os.remove(temp_raw_path)


In [30]:
# Process Your Video File
# Define input and output paths
video_path = '/Users/stupam2/Desktop/project/cv2/test1.mp4'
output_path = '/Users/stupam2/Desktop/project/cv2/depth_test1.mp4'

# Generate depth video
generate_depth_video(video_path, output_path)


INFO:root:Processing 427 frames at 60.0 FPS with resolution 1920x1080.
INFO:root:Processed 10/427 frames.
INFO:root:Processed 20/427 frames.
INFO:root:Processed 30/427 frames.
INFO:root:Processed 40/427 frames.
INFO:root:Processed 50/427 frames.
INFO:root:Processed 60/427 frames.
INFO:root:Processed 70/427 frames.
INFO:root:Processed 80/427 frames.
INFO:root:Processed 90/427 frames.
INFO:root:Processed 100/427 frames.
INFO:root:Processed 110/427 frames.
INFO:root:Processed 120/427 frames.
INFO:root:Processed 130/427 frames.
INFO:root:Processed 140/427 frames.
INFO:root:Processed 150/427 frames.
INFO:root:Processed 160/427 frames.
INFO:root:Processed 170/427 frames.
INFO:root:Processed 180/427 frames.
INFO:root:Processed 190/427 frames.
INFO:root:Processed 200/427 frames.
INFO:root:Processed 210/427 frames.
INFO:root:Processed 220/427 frames.
INFO:root:Processed 230/427 frames.
INFO:root:Processed 240/427 frames.
INFO:root:Processed 250/427 frames.
INFO:root:Processed 260/427 frames.
IN

HTTPException: 

In [19]:
# Initialize Logging and Load the MiDaS Model
# Set up logging
logging.basicConfig(level=logging.INFO)

# Load the MiDaS model from PyTorch Hub
model_type = "DPT_Hybrid"  # Specify the model type; can also be DPT_Large, DPT_Hybrid, etc.
model = torch.hub.load("intel-isl/MiDaS", model_type)
model.eval()
device = torch.device('cpu')  # Use CPU
model.to(device)


Using cache found in /Users/stupam2/.cache/torch/hub/intel-isl_MiDaS_master
/opt/homebrew/Caskroom/miniconda/base/envs/chatbot_env/lib/python3.9/site-packages/timm/models/_factory.py:117: UserWarning: Mapping deprecated model name vit_base_resnet50_384 to current vit_base_r50_s16_384.orig_in21k_ft_in1k.
  model = create_fn(


DPTDepthModel(
  (pretrained): Module(
    (model): VisionTransformer(
      (patch_embed): HybridEmbed(
        (backbone): ResNetV2(
          (stem): Sequential(
            (conv): StdConv2dSame(3, 64, kernel_size=(7, 7), stride=(2, 2), bias=False)
            (norm): GroupNormAct(
              32, 64, eps=1e-05, affine=True
              (drop): Identity()
              (act): ReLU(inplace=True)
            )
            (pool): MaxPool2dSame(kernel_size=(3, 3), stride=(2, 2), padding=(0, 0), dilation=(1, 1), ceil_mode=False)
          )
          (stages): Sequential(
            (0): ResNetStage(
              (blocks): Sequential(
                (0): Bottleneck(
                  (downsample): DownsampleConv(
                    (conv): StdConv2dSame(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
                    (norm): GroupNormAct(
                      32, 256, eps=1e-05, affine=True
                      (drop): Identity()
                      (act): Identit

In [20]:
# Define the Image Transformation
# Define the transformation required for the MiDaS input
transform = Compose([
    Resize((384, 384)),  # Resize to 384x384 pixels
    ToTensor(),  # Convert the image to a tensor
    Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])


In [21]:
# Define the Depth Estimation Function
def estimate_depth(frame):
    """Estimate the depth of a frame using the MiDaS model."""
    try:
        pil_image = Image.fromarray(frame)
        input_tensor = transform(pil_image).unsqueeze(0).to(device)
        with torch.no_grad():
            depth_map = model(input_tensor)
        # Get the original size to resize depth map
        original_size = (frame.shape[1], frame.shape[0])  # (width, height)
        depth_map = torch.nn.functional.interpolate(
            depth_map.unsqueeze(1),
            size=original_size,  # Resize depth map to original frame size
            mode='bicubic',
            align_corners=False
        ).squeeze().cpu().numpy()
        return depth_map
    except Exception as e:
        logging.error(f"Error estimating depth: {e}")
        raise


In [22]:
# Define the Function to Generate the Depth Video
def generate_depth_video(video_path, output_path):
    try:
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            logging.error("Error: Could not open video file.")
            return

        frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = cap.get(cv2.CAP_PROP_FPS)

        logging.info(f"Processing {frame_count} frames at {fps} FPS with resolution {frame_width}x{frame_height}.")

        # Define the codec and create VideoWriter object for temporary raw output
        temp_raw_path = "temp_raw.avi"
        raw_out = cv2.VideoWriter(temp_raw_path, cv2.VideoWriter_fourcc(*'MJPG'), fps, (frame_width, frame_height), isColor=True)

        processed_frames = 0
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                logging.info("Finished processing video.")
                break
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            depth_map = estimate_depth(frame_rgb)
            depth_map_normalized = cv2.normalize(depth_map, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
            depth_colormap = cv2.applyColorMap(depth_map_normalized, cv2.COLORMAP_JET)
            raw_out.write(depth_colormap)
            processed_frames += 1

            # Log every 10th frame to track progress
            if processed_frames % 10 == 0:
                logging.info(f"Processed {processed_frames}/{frame_count} frames.")

        cap.release()
        raw_out.release()

        logging.info(f"Processed {processed_frames} frames.")

        # Verify the raw output file size
        if os.path.exists(temp_raw_path) and os.path.getsize(temp_raw_path) > 0:
            # Use ffmpeg to convert raw video to QuickTime compatible format
            command = [
                'ffmpeg', '-y', '-i', temp_raw_path,
                '-c:v', 'libx264', '-pix_fmt', 'yuv420p',
                output_path
            ]
            subprocess.run(command, check=True)
            logging.info(f"Converted video saved to {output_path}.")
        else:
            logging.error("No frames were written to the temporary raw file.")
            raise HTTPException(status_code=500, detail="Failed to write frames to raw video.")

    except subprocess.CalledProcessError as e:
        logging.error(f"FFmpeg error: {e}")
        raise HTTPException(status_code=500, detail="Error during video conversion.")
    except Exception as e:
        logging.error(f"Error generating depth video: {e}")
        raise
    finally:
        if os.path.exists(temp_raw_path):
            os.remove(temp_raw_path)


In [23]:
# Process Your Video File
# Define input and output paths
video_path = '/Users/stupam2/Desktop/project/cv2/test1.mp4'
output_path = '/Users/stupam2/Desktop/project/cv2/depth_test1.mp4'

# Generate depth video
generate_depth_video(video_path, output_path)


INFO:root:Processing 427 frames at 60.0 FPS with resolution 1920x1080.
INFO:root:Processed 10/427 frames.
INFO:root:Processed 20/427 frames.
INFO:root:Processed 30/427 frames.
INFO:root:Processed 40/427 frames.
INFO:root:Processed 50/427 frames.
INFO:root:Processed 60/427 frames.
INFO:root:Processed 70/427 frames.
INFO:root:Processed 80/427 frames.
INFO:root:Processed 90/427 frames.
INFO:root:Processed 100/427 frames.
INFO:root:Processed 110/427 frames.
INFO:root:Processed 120/427 frames.
INFO:root:Processed 130/427 frames.
INFO:root:Processed 140/427 frames.
INFO:root:Processed 150/427 frames.
INFO:root:Processed 160/427 frames.
INFO:root:Processed 170/427 frames.
INFO:root:Processed 180/427 frames.
INFO:root:Processed 190/427 frames.
INFO:root:Processed 200/427 frames.
INFO:root:Processed 210/427 frames.
INFO:root:Processed 220/427 frames.
INFO:root:Processed 230/427 frames.
INFO:root:Processed 240/427 frames.
INFO:root:Processed 250/427 frames.
INFO:root:Processed 260/427 frames.
IN

HTTPException: 

In [10]:
# Cell 2: Initialize FastAPI and Set Up Logging
# Initialize the FastAPI app
app = FastAPI()

# Set up logging
logging.basicConfig(level=logging.INFO)


In [11]:
# Cell 3: Load the MiDaS Model
# Load the MiDaS model from PyTorch Hub
model_type = "DPT_Hybrid"  # Specify the model type; can also be DPT_Large, DPT_Hybrid, etc.
model = torch.hub.load("intel-isl/MiDaS", model_type)
model.eval()
device = torch.device('cpu')  # Use CPU
model.to(device)


Using cache found in /Users/stupam2/.cache/torch/hub/intel-isl_MiDaS_master


DPTDepthModel(
  (pretrained): Module(
    (model): VisionTransformer(
      (patch_embed): HybridEmbed(
        (backbone): ResNetV2(
          (stem): Sequential(
            (conv): StdConv2dSame(3, 64, kernel_size=(7, 7), stride=(2, 2), bias=False)
            (norm): GroupNormAct(
              32, 64, eps=1e-05, affine=True
              (drop): Identity()
              (act): ReLU(inplace=True)
            )
            (pool): MaxPool2dSame(kernel_size=(3, 3), stride=(2, 2), padding=(0, 0), dilation=(1, 1), ceil_mode=False)
          )
          (stages): Sequential(
            (0): ResNetStage(
              (blocks): Sequential(
                (0): Bottleneck(
                  (downsample): DownsampleConv(
                    (conv): StdConv2dSame(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
                    (norm): GroupNormAct(
                      32, 256, eps=1e-05, affine=True
                      (drop): Identity()
                      (act): Identit

In [12]:
# Cell 4: Define the Image Transformation
# Define the transformation required for the MiDaS input
transform = Compose([
    Resize((384, 384)),  # Resize to 384x384 pixels
    ToTensor(),  # Convert the image to a tensor
    Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])


In [13]:
# Cell 5: Define the Depth Estimation Function
def estimate_depth(frame):
    """Estimate the depth of a frame using the MiDaS model."""
    try:
        pil_image = Image.fromarray(frame)
        input_tensor = transform(pil_image).unsqueeze(0).to(device)
        with torch.no_grad():
            depth_map = model(input_tensor)
        # Get the original size to resize depth map
        original_size = (frame.shape[1], frame.shape[0])  # (width, height)
        depth_map = torch.nn.functional.interpolate(
            depth_map.unsqueeze(1),
            size=original_size,  # Resize depth map to original frame size
            mode='bicubic',
            align_corners=False
        ).squeeze().cpu().numpy()
        return depth_map
    except Exception as e:
        logging.error(f"Error estimating depth: {e}")
        raise


In [14]:
# Cell 6: Define the Function to Generate the Depth Video
def generate_depth_video(video_path, output_path):
    try:
        cap = cv2.VideoCapture(video_path)
        frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = cap.get(cv2.CAP_PROP_FPS)

        logging.info(f"Processing {frame_count} frames at {fps} FPS with resolution {frame_width}x{frame_height}.")

        # Define the codec and create VideoWriter object for temporary raw output
        temp_raw_path = "temp_raw.avi"
        raw_out = cv2.VideoWriter(temp_raw_path, cv2.VideoWriter_fourcc(*'MJPG'), fps, (frame_width, frame_height), isColor=True)

        processed_frames = 0
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                logging.info("Finished processing video.")
                break
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            depth_map = estimate_depth(frame_rgb)
            depth_map_normalized = cv2.normalize(depth_map, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
            depth_colormap = cv2.applyColorMap(depth_map_normalized, cv2.COLORMAP_JET)
            raw_out.write(depth_colormap)
            processed_frames += 1
        cap.release()
        raw_out.release()

        logging.info(f"Processed {processed_frames} frames.")

        # Verify the raw output file size
        if os.path.exists(temp_raw_path) and os.path.getsize(temp_raw_path) > 0:
            # Use ffmpeg to convert raw video to QuickTime compatible format
            command = [
                'ffmpeg', '-y', '-i', temp_raw_path,
                '-c:v', 'libx264', '-pix_fmt', 'yuv420p',
                output_path
            ]
            subprocess.run(command, check=True)
            logging.info(f"Converted video saved to {output_path}.")
        else:
            logging.error("No frames were written to the temporary raw file.")
            raise HTTPException(status_code=500, detail="Failed to write frames to raw video.")

    except subprocess.CalledProcessError as e:
        logging.error(f"FFmpeg error: {e}")
        raise HTTPException(status_code=500, detail="Error during video conversion.")
    except Exception as e:
        logging.error(f"Error generating depth video: {e}")
        raise
    finally:
        if os.path.exists(temp_raw_path):
            os.remove(temp_raw_path)


In [15]:
# Cell 7: Define the API Endpoint
@app.post("/depth-map/")
async def depth_map_endpoint(file: UploadFile = File(...)):
    try:
        video_path = f"temp_{file.filename}"
        output_path = f"depth_{file.filename}.mp4"
        with open(video_path, "wb") as buffer:
            buffer.write(await file.read())

        generate_depth_video(video_path, output_path)

        # Check if output file exists and is not empty before sending response
        if os.path.exists(output_path) and os.path.getsize(output_path) > 0:
            return FileResponse(output_path, media_type='video/mp4', filename=f"depth_{file.filename}")
        else:
            raise HTTPException(status_code=500, detail="Failed to generate video.")
    except Exception as e:
        logging.error(f"Error processing request: {e}")
        raise HTTPException(status_code=500, detail="Internal Server Error")
    finally:
        # Clean up temporary files
        if os.path.exists(video_path):
            os.remove(video_path)
        if os.path.exists(output_path):
            os.remove(output_path)


In [ ]:
# Cell 8: Run the FastAPI App in Jupyter
import nest_asyncio
from threading import Thread

# Apply the asyncio patch
nest_asyncio.apply()

# Function to run Uvicorn server in a separate thread
def run_app():
    uvicorn.run(app, host="0.0.0.0", port=8000)

# Start the server in a new thread
thread = Thread(target=run_app)
thread.start()


INFO:     Started server process [27695]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


In [17]:
# Process Your Video File
# Define input and output paths
video_path = '/Users/stupam2/Desktop/project/cv2/test1.mp4'
output_path = '/Users/stupam2/Desktop/project/cv2/depth_test1.mp4'

# Generate depth video
generate_depth_video(video_path, output_path)


INFO:root:Processing 427 frames at 60.0 FPS with resolution 1920x1080.
INFO:root:Finished processing video.
INFO:root:Processed 427 frames.
ffmpeg version 7.0.2 Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 15.0.0 (clang-1500.3.9.4)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.0.2 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable

HTTPException: 